In [ ]:
import os
import shutil
import cv2
from PIL import Image
import torch
import torch.nn as nn
from torchvision import models, transforms

# Load the trained model
def load_model(model_path):
    model = models.resnet18(pretrained=False)  # We initialize ResNet18 without pre-trained weights
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 2)  # Adjust for binary classification (User POV and Player POV)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
    return model

# Function to extract the first frame from a video
def extract_first_frame_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Failed to open video file: {video_path}")
        return None
    
    # Read the first frame
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
        return Image.fromarray(frame)
    else:
        print(f"Failed to extract first frame from: {video_path}")
        return None

# Function to classify the first frame of the video
def predict_video_first_frame(video_path, model, device):
    model.eval()
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    
    frame = extract_first_frame_from_video(video_path)
    if frame is None:
        return None
    
    image = transform(frame).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        predicted_class = class_names[preds[0]]
    
    return predicted_class


# Function to classify and move video files using os.walk()
def classify_and_move_videos(input_folder, pov_user_folder, pov_player_folder, model, device):
    # Ensure the POV user and POV player folders exist
    if not os.path.exists(pov_user_folder):
        os.makedirs(pov_user_folder)
    if not os.path.exists(pov_player_folder):
        os.makedirs(pov_player_folder)

    # Traverse through all directories and subdirectories using os.walk
    for root, dirs, files in os.walk(input_folder):
        # Extract the folder name (e.g., '2024-09-26 21-58-06') from the current directory
        folder_name = os.path.basename(root)

        # Iterate through video files in the current directory
        for video_file in files:
            if video_file.endswith(('.mp4', '.avi', '.mkv', '.mov')):  # Add video formats you are using
                video_path = os.path.join(root, video_file)

                # Classify the video based on the first frame
                prediction = predict_video_first_frame(video_path, model, device)
                print(f"Prediction for {video_file}: {prediction}")
                if prediction is None:
                    continue

                # Determine the destination folder based on the prediction
                if prediction == 'POV user':  # Assuming class names are 'POV user' and 'POV player'
                    destination_folder = os.path.join(pov_user_folder, folder_name)  # Folder like 'POV user\2024-09-26 21-58-06'
                elif prediction == 'POV player':
                    destination_folder = os.path.join(pov_player_folder, folder_name)  # Folder like 'POV player\2024-09-26 21-58-06'
                else:
                    print(f"Unknown prediction for {video_file}")
                    continue

                # Ensure the destination folder exists
                if not os.path.exists(destination_folder):
                    os.makedirs(destination_folder)

                # Copy the file to the destination folder, keeping the original name
                destination = os.path.join(destination_folder, video_file)
                shutil.copy2(video_path, destination)
                print(f"Copied {video_file} to {destination}")


# Main code
if __name__ == "__main__":
    # Define paths
    input_folder = r'path to\output'  # Folder with your video clips
    pov_user_folder = r'path to\clip/POV user'  # Folder to store classified User POV videos
    pov_player_folder = r'path to\clip/POV player'  # Folder to store classified Player POV videos
    model_path = 'valorant_pov_classifier.pth'  # Path to your saved model file

    # Load the model
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = load_model(model_path).to(device)

    # Class names
    class_names = ['POV player', 'POV user' ]  # Update these according to your classification labels

    # Classify and move videos
    classify_and_move_videos(input_folder, pov_user_folder, pov_player_folder, model,device)
